In [3]:
import pandas as pd
import numpy as np
import datetime as dt
import time
import bs4 as bs
import re
import pickle
import os
import requests
import matplotlib.ticker as ticker
from matplotlib.dates import date2num, DayLocator, DateFormatter
pd.core.common.is_list_like = pd.api.types.is_list_like
import pandas_datareader as web

# Data preparation

Using data from the S&P 500, I want to analyze the companies to see if I can find some surface level patterns to group companies together. After which, I will use historical data collected from yahoo finance to set up technical indicators and conduct quantitative analysis to find any useful patterns to set up buy and sell signals. 

I'll start by scraping the S&P 500 companies list from the wikipedia page and load in the tickers through yahoo finance. After which, I will cluster their returns on top of other indicators such as sectory and market cap to find any salient groupings. 

In [4]:
start = '2017-01-01'
end = '2019-03-25'

Unfortunately somehwere along the way, the data from yahoo finance stopped loading due to errors in the API, then I switched to Quandl. However, quandl also started giving missing data quite often when there should have been data there as well. Due to this, I am reverting to using a precompiled source of S&P500 company data from kaggle which ranges from 2013 to Feb 2018. For the sake of simplicity, I will use this to make the dataset for the clustering portion, but at one point this code worked as intended. If fixes are made to these APIs, then they can be used to scrape custom csv files with differing variables.

In [5]:
# scraping all the company tickers from the S&P 500
def save_sp500_companies():
    resp = requests.get('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
    soup = bs.BeautifulSoup(resp.text, 'lxml')
    table = soup.find('table', {'class': 'wikitable sortable'})
    tickers = []
    for row in table.findAll('tr')[1:]:
        ticker = row.findAll('td')[1].text
        #ticker = ticker.replace('.','') Kaggle data accounts for dot notation used in stock listings
        tickers.append(ticker)
    # saves the data into a local format so I don't have to reload it everytime
    with open('sp500tickers.pickle', 'wb') as f:
        pickle.dump(tickers, f)
        
    print(tickers)
    return tickers

In [6]:
def get_data_quandl(reload_sp500=False):
    if reload_sp500:
        tickers = save_sp500_companies()        
    else:
        with open('sp500tickers.pickle', 'rb') as f:
            tickers = pickle.load(f)
            
    if not os.path.exists('stock_dfs'):
        os.makedirs('stock_dfs')
# load in the tickers and make them into one dataframe
    for ticker in tickers[:]:
        if not os.path.exists('stock_dfs/{}.csv'.format(ticker)):
            try:
                df = web.DataReader('WIKI/{}'.format(ticker), 'quandl', start, end,access_key='qaB2Gz6zLzLLoHroWJUF').loc[::-1, ['AdjClose', 'Volume']]
                df.to_csv('stock_dfs/{}.csv'.format(ticker))
            except Exception as e:
                print("{} not loading. ".format(ticker), e)
        else:
            print("{} already exists.".format(ticker))
            
get_data_quandl(reload_sp500=False)

MMM already exists.
ABT already exists.
ABBV already exists.
ABMD already exists.
ACN already exists.
ATVI already exists.
ADBE already exists.
AMD already exists.
AAP already exists.
AES already exists.
AMG already exists.
AFL already exists.
A already exists.
APD already exists.
AKAM already exists.
ALK already exists.
ALB already exists.
ARE already exists.
ALXN already exists.
ALGN already exists.
ALLE already exists.
AGN already exists.
ADS already exists.
LNT already exists.
ALL already exists.
GOOGL already exists.
GOOG already exists.
MO already exists.
AMZN already exists.
AEE already exists.
AAL already exists.
AEP already exists.
AXP already exists.
AIG already exists.
AMT already exists.
AWK already exists.
AMP already exists.
ABC already exists.
AME already exists.
AMGN already exists.
APH already exists.
APC already exists.
ADI already exists.
ANSS already exists.
ANTM already exists.
AON already exists.
AOS already exists.
APA already exists.
AIV already exists.
AAPL alr

PKG already exists.
PH already exists.
PAYX already exists.
PYPL already exists.
PNR already exists.
PBCT already exists.
PEP already exists.
PKI already exists.
PRGO already exists.
PFE already exists.
PM already exists.
PSX already exists.
PNW already exists.
PXD already exists.
PNC already exists.
RL already exists.
PPG already exists.
PPL already exists.
PFG already exists.
PG already exists.
PGR already exists.
PLD already exists.
PRU already exists.
PEG already exists.
PSA already exists.
PHM already exists.
PVH already exists.
QRVO already exists.
PWR already exists.
QCOM already exists.
DGX already exists.
RJF already exists.
RTN already exists.
O already exists.
RHT already exists.
REG already exists.
REGN already exists.
RF already exists.
RSG already exists.
RMD already exists.
RHI already exists.
ROK already exists.
ROL already exists.
ROST already exists.
RCL already exists.
CRM already exists.
SBAC already exists.
SLB already exists.
STX already exists.
SEE already exists

In [7]:
def compile_dfs():
    with open('sp500tickers.pickle', 'rb') as f:
        tickers = pickle.load(f)
    prices = []
    prices_df = pd.DataFrame()
    for count, ticker in enumerate(tickers):
        if count % 10 == 0 :
            print(count)
        try:
            df = pd.read_csv('stock_dfs/{}.csv'.format(ticker))
            df = df.rename(columns={'AdjClose': ticker})
            df.index = df.Date
            df.drop('Date', axis=1, inplace=True)
            df = df[ticker]
        except Exception as e:
            print('{} not present.'.format(ticker), e)
        
        prices.append(df)
        prices_df = pd.concat([prices_df, df], axis=1)
            
    print(prices_df.head())
    prices_df.to_csv('sp500_close_joined.csv')
        
compile_dfs()

0
10
20
30
40
50
ANET not present. File b'stock_dfs/ANET.csv' does not exist
60
70
BRK.B not present. File b'stock_dfs/BRK.B.csv' does not exist
80
BF.B not present. File b'stock_dfs/BF.B.csv' does not exist
90
CPRI not present. File b'stock_dfs/CPRI.csv' does not exist
100
110
120
130
140
150
160
170
180
EVRG not present. File b'stock_dfs/EVRG.csv' does not exist
190
200
210
220
230
240
250
260
JEF not present. File b'stock_dfs/JEF.csv' does not exist
270
KEYS not present. File b'stock_dfs/KEYS.csv' does not exist
280
LW not present. File b'stock_dfs/LW.csv' does not exist
290
LIN not present. File b'stock_dfs/LIN.csv' does not exist
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
                   MMM        ABT       ABBV    ABMD         ACN       ATVI  \
2017-01-03  174.831943  38.170443  60.192574  112.36  114.153742  36.420423   
2017-01-04  175.097063  38.473461  61.041308  115.74  114.428197  37.136109   
2017-01-05  174.498088  38.805803  6

### Creating dataset from kaggle data if yahoo finance is not working

In [8]:
def compile_sp500_close_kaggle(reload_sp500=True):
    if reload_sp500:
        tickers = save_sp500_companies()        
    else:
        with open('sp500tickers.pickle', 'rb') as f:
            tickers = pickle.load(f)
    prices = []
    prices_df = pd.DataFrame()
    for count, ticker in enumerate(tickers):
        if (count %10 == 0):
            print(count) #show progress
        
        try:
            df = pd.read_csv('individual_stocks_5yr/{}_data.csv'.format(ticker), index_col=0)
            df = df.rename(columns={'close': ticker})
            df = df[ticker]
        except Exception as e:
            print('{} not loaded'.format(ticker), e)
        
        prices.append(df)
        prices_df = pd.concat([prices_df, df], axis=1)
        
    print(prices_df.head())
    prices_df.to_csv('sp500_close_joined.csv')
compile_sp500_close_kaggle()

['MMM', 'ABT', 'ABBV', 'ABMD', 'ACN', 'ATVI', 'ADBE', 'AMD', 'AAP', 'AES', 'AMG', 'AFL', 'A', 'APD', 'AKAM', 'ALK', 'ALB', 'ARE', 'ALXN', 'ALGN', 'ALLE', 'AGN', 'ADS', 'LNT', 'ALL', 'GOOGL', 'GOOG', 'MO', 'AMZN', 'AEE', 'AAL', 'AEP', 'AXP', 'AIG', 'AMT', 'AWK', 'AMP', 'ABC', 'AME', 'AMGN', 'APH', 'APC', 'ADI', 'ANSS', 'ANTM', 'AON', 'AOS', 'APA', 'AIV', 'AAPL', 'AMAT', 'APTV', 'ADM', 'ARNC', 'ANET', 'AJG', 'AIZ', 'ATO', 'T', 'ADSK', 'ADP', 'AZO', 'AVB', 'AVY', 'BHGE', 'BLL', 'BAC', 'BK', 'BAX', 'BBT', 'BDX', 'BRK.B', 'BBY', 'BIIB', 'BLK', 'HRB', 'BA', 'BKNG', 'BWA', 'BXP', 'BSX', 'BHF', 'BMY', 'AVGO', 'BR', 'BF.B', 'CHRW', 'COG', 'CDNS', 'CPB', 'COF', 'CPRI', 'CAH', 'KMX', 'CCL', 'CAT', 'CBOE', 'CBRE', 'CBS', 'CE', 'CELG', 'CNC', 'CNP', 'CTL', 'CERN', 'CF', 'SCHW', 'CHTR', 'CVX', 'CMG', 'CB', 'CHD', 'CI', 'XEC', 'CINF', 'CTAS', 'CSCO', 'C', 'CFG', 'CTXS', 'CLX', 'CME', 'CMS', 'KO', 'CTSH', 'CL', 'CMCSA', 'CMA', 'CAG', 'CXO', 'COP', 'ED', 'STZ', 'COO', 'CPRT', 'GLW', 'COST', 'COTY', 'CC